<a href="https://colab.research.google.com/github/rikdantas/Aprendizagem-de-Maquinas/blob/main/IMD1101/Pratica_03/Pratica03_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prática 03 - Técnica Supervisionada (NB)
Nesse notebook será aplicado o método de Naive Bayes nas 6 melhores bases que eu tinha encontrado na prática 01 e também nas 6 bases que foram aplicados o PCA também na prática 01.

Aluno: Paulo Ricardo Dantas

In [1]:
# Importando as bibliotecas
import numpy as np
import pandas as pd
import gdown
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

## Baixando os datasets

In [2]:
# Criando função para baixar os datasets, assim deixando o código um pouco mais organizado
def baixar_dataset(url, output):
    gdown.download(url, output, quiet=True)

In [3]:
# Baixando datasets do VGG16
baixar_dataset('https://drive.google.com/uc?id=1Lig0-UqpGvMQCmV5r5U__yucoocrlbtf', 'VGG16_1.csv')
baixar_dataset('https://drive.google.com/uc?id=1V7_N11eqBnTiM9iKY3vhMVrTzoERrijS', 'VGG16_2.csv')
baixar_dataset('https://drive.google.com/uc?id=1d8spY-o4xBsozdeSbGGlweZrips1m0HC', 'VGG16_3.csv')

# Baixando datasets do VGG19
baixar_dataset('https://drive.google.com/uc?id=1yjuqQKxXtBqlMetGgD3fyFjDZ3zg3YDS', 'VGG19_1.csv')
baixar_dataset('https://drive.google.com/uc?id=13nmb2kPrv6oVKlhodM-_ierzsMrvWL18', 'VGG19_2.csv')
baixar_dataset('https://drive.google.com/uc?id=1qB2SQXLtgS01FqTrwGYLdOY1pwRqkuXu', 'VGG19_3.csv')

# Baixando datasets do PCA
baixar_dataset('https://drive.google.com/uc?id=14sGLGGak-1WZwnLrqEMEZbYWNZL1dIVn', 'PCA_1.csv')
baixar_dataset('https://drive.google.com/uc?id=1DCXErsoiqwcM59QkIOdlZcIVGZDmWYTK', 'PCA_2.csv')
baixar_dataset('https://drive.google.com/uc?id=15k9bMm1ZfliNeHQ6MX7SlEy49UQJrxtv', 'PCA_3.csv')
baixar_dataset('https://drive.google.com/uc?id=17BJNdaH_8fCYTJn-l-QH7ayRaLuWYxQ6', 'PCA_4.csv')
baixar_dataset('https://drive.google.com/uc?id=18_qRzZ4wA-laBL0bsCch6xTus7SYLjLf', 'PCA_5.csv')
baixar_dataset('https://drive.google.com/uc?id=1mc9uT_EDkh9xC8rWnZTUZp6qX8Q7verz', 'PCA_6.csv')

## Importando os datasets
Vamos usar o pandas para importar os datasets para DataFrames. Usaremos os mesmos nomes que foram baixados os arquivos dos datasets. Podemos relacionar os datasets com as configurações como visto na tabela a seguir:

| Nome do dataset | Configuração            |
|-----------------|-------------------------|
| VGG16_1         | CNN_VGG16_128_avg       |
| VGG16_2         | CNN_VGG16_128_max       |
| VGG16_3         | CNN_VGG16_256_avg       |
| VGG19_1         | CNN_VGG19_128_avg       |
| VGG19_2         | CNN_VGG19_128_max       |
| VGG19_3         | CNN_VGG19_256_avg       |
| PCA_1           | PCA_CNN_VGG16_128_avg   |
| PCA_2           | PCA_CNN_VGG16_128_max   |
| PCA_3           | PCA_CNN_VGG16_256_avg   |
| PCA_4           | PCA_CNN_VGG19_128_avg   |
| PCA_5           | PCA_CNN_VGG19_128_max   |
| PCA_6           | PCA_CNN_VGG19_256_avg   |


In [4]:
# Importando os datasets do VGG16
vgg16_1 = pd.read_csv('VGG16_1.csv', encoding='utf-8')
vgg16_2 = pd.read_csv('VGG16_2.csv', encoding='utf-8')
vgg16_3 = pd.read_csv('VGG16_3.csv', encoding='utf-8')

# Importando os datasets do VGG19
vgg19_1 = pd.read_csv('VGG19_1.csv', encoding='utf-8')
vgg19_2 = pd.read_csv('VGG19_2.csv', encoding='utf-8')
vgg19_3 = pd.read_csv('VGG19_3.csv', encoding='utf-8')

# Importando os datasets do PCA
pca_1 = pd.read_csv('PCA_1.csv', encoding='utf-8')
pca_2 = pd.read_csv('PCA_2.csv', encoding='utf-8')
pca_3 = pd.read_csv('PCA_3.csv', encoding='utf-8')
pca_4 = pd.read_csv('PCA_4.csv', encoding='utf-8')
pca_5 = pd.read_csv('PCA_5.csv', encoding='utf-8')
pca_6 = pd.read_csv('PCA_6.csv', encoding='utf-8')

## Aplicando o Naive Bayes


### Utilizando o holdout

In [5]:
# Lista de datasets
datasets = {
    'vgg16_1': vgg16_1, 'vgg16_2': vgg16_2, 'vgg16_3': vgg16_3,
    'vgg19_1': vgg19_1, 'vgg19_2': vgg19_2, 'vgg19_3': vgg19_3,
    'pca_1': pca_1, 'pca_2': pca_2, 'pca_3': pca_3,
    'pca_4': pca_4, 'pca_5': pca_5, 'pca_6': pca_6
}

# Modelos de Naive Bayes
models = {
    'GaussianNB': GaussianNB(),
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB()
}

# Resultados
results = []

In [6]:
for dataset_name, dataset in datasets.items():
    X = dataset.iloc[:, :-1]  # Todas as colunas menos a última
    y = dataset.iloc[:, -1]   # Última coluna (rótulo)

    # Divisão holdout
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

    for model_name, model in models.items():
        try:
            # Treinar o modelo
            model.fit(X_train, y_train)
            # Predições
            y_pred = model.predict(X_test)
            # Calcular acurácia
            accuracy = accuracy_score(y_test, y_pred)
            # Armazenar o resultado
            results.append({
                'Dataset': dataset_name,
                'Model': model_name,
                'Accuracy': accuracy
            })
        except Exception as e:
            results.append({
                'Dataset': dataset_name,
                'Model': model_name,
                'Error': str(e)
            })

# Convertendo os resultados para dataframe
results_df = pd.DataFrame(results)

# Salvando os resultados em CSV
results_df.to_csv('Resultados_Naive_Bayes_Holdout.csv', index=False)

# Mostrando o resultado
print(results_df)

    Dataset          Model  Accuracy  \
0   vgg16_1     GaussianNB  0.475000   
1   vgg16_1  MultinomialNB  0.483333   
2   vgg16_1   ComplementNB  0.608333   
3   vgg16_2     GaussianNB  0.479167   
4   vgg16_2  MultinomialNB  0.658333   
5   vgg16_2   ComplementNB  0.654167   
6   vgg16_3     GaussianNB  0.600000   
7   vgg16_3  MultinomialNB  0.479167   
8   vgg16_3   ComplementNB  0.479167   
9   vgg19_1     GaussianNB  0.475000   
10  vgg19_1  MultinomialNB  0.479167   
11  vgg19_1   ComplementNB  0.533333   
12  vgg19_2     GaussianNB  0.475000   
13  vgg19_2  MultinomialNB  0.612500   
14  vgg19_2   ComplementNB  0.620833   
15  vgg19_3     GaussianNB  0.575000   
16  vgg19_3  MultinomialNB  0.479167   
17  vgg19_3   ComplementNB  0.479167   
18    pca_1     GaussianNB  0.633333   
19    pca_1  MultinomialNB       NaN   
20    pca_1   ComplementNB       NaN   
21    pca_2     GaussianNB  0.620833   
22    pca_2  MultinomialNB       NaN   
23    pca_2   ComplementNB       NaN   


### Utilizando K-Fold

In [7]:
# Modelos de Naive Bayes
models = {
    'GaussianNB': GaussianNB(),
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB()
}

# Lista de datasets
datasets = {
    'vgg16_1': vgg16_1, 'vgg16_2': vgg16_2, 'vgg16_3': vgg16_3,
    'vgg19_1': vgg19_1, 'vgg19_2': vgg19_2, 'vgg19_3': vgg19_3,
    'pca_1': pca_1, 'pca_2': pca_2, 'pca_3': pca_3,
    'pca_4': pca_4, 'pca_5': pca_5, 'pca_6': pca_6
}

# Resultados para k-fold
results_kfold = []

In [8]:
kf = KFold(n_splits=10, shuffle=True, random_state=1)

for dataset_name, dataset in datasets.items():
    X = dataset.iloc[:, :-1]  # Todas as colunas menos a última
    y = dataset.iloc[:, -1]   # Última coluna (rótulo)

    for model_name, model in models.items():
        fold_accuracies = []

        for train_index, test_index in kf.split(X):
            # Dividir os dados em treino e teste
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            try:
                # Treinar o modelo
                model.fit(X_train, y_train)
                # Predições
                y_pred = model.predict(X_test)
                # Calcular acurácia
                accuracy = accuracy_score(y_test, y_pred)
                fold_accuracies.append(accuracy)
            except Exception as e:
                fold_accuracies.append(f"Error: {str(e)}")

        # Calcular a média das acurácias das folds
        if all(isinstance(acc, float) for acc in fold_accuracies):
            mean_accuracy = sum(fold_accuracies) / len(fold_accuracies)
            results_kfold.append({
                'Dataset': dataset_name,
                'Model': model_name,
                'Mean Accuracy': mean_accuracy
            })
        else:
            results_kfold.append({
                'Dataset': dataset_name,
                'Model': model_name,
                'Error': fold_accuracies
            })

# Convertendo os resultados para dataframe
results_kfold_df = pd.DataFrame(results_kfold)

# Salvando os resultados em CSV
results_kfold_df.to_csv('Resultados_Naive_Bayes_KFOLD.csv', index=False)

# Mostrando o resultado
print(results_kfold_df)

    Dataset          Model  Mean Accuracy  \
0   vgg16_1     GaussianNB        0.64875   
1   vgg16_1  MultinomialNB        0.61125   
2   vgg16_1   ComplementNB        0.65000   
3   vgg16_2     GaussianNB        0.51750   
4   vgg16_2  MultinomialNB        0.67750   
5   vgg16_2   ComplementNB        0.68000   
6   vgg16_3     GaussianNB        0.61500   
7   vgg16_3  MultinomialNB        0.54000   
8   vgg16_3   ComplementNB        0.58375   
9   vgg19_1     GaussianNB        0.63875   
10  vgg19_1  MultinomialNB        0.57625   
11  vgg19_1   ComplementNB        0.60500   
12  vgg19_2     GaussianNB        0.60125   
13  vgg19_2  MultinomialNB        0.63750   
14  vgg19_2   ComplementNB        0.63625   
15  vgg19_3     GaussianNB        0.56000   
16  vgg19_3  MultinomialNB        0.48875   
17  vgg19_3   ComplementNB        0.50500   
18    pca_1     GaussianNB        0.65750   
19    pca_1  MultinomialNB            NaN   
20    pca_1   ComplementNB            NaN   
21    pca_